# 7차시: 텐서플로우 전이학습

## 2023. 08. 16. 14:10 ~ 16:00 (50분×2)
1. 전이학습 기초
1. CIFAR-10 데이터 셋
1. 텐서플로우 실습

### 참고자료
- [파이썬 3 표준 문서](https://docs.python.org/3/index.html)
- [텐서플로우 이미지 분류](https://www.tensorflow.org/tutorials/keras/classification)
- [텐서플로우 전이학습](https://www.tensorflow.org/tutorials/images/transfer_learning_with_hub)

### 1. 도구 불러오기 및 버전 확인

In [ ]:
# 도구 준비
import os
import random

import tensorflow as tf # 텐서플로우
import tensorflow_hub as hub
import matplotlib.pyplot as plt # 시각화 도구
%matplotlib inline
import numpy as np
import PIL.Image as Image

In [ ]:
print(f'Tensorflow 버전을 확인합니다: {tf.__version__}')

### 2. 학습 데이터 불러오기

In [ ]:
# prepare dataset
dataset_root = os.path.abspath(os.path.expanduser('/tmp/dataset'))
print(f'Dataset root: {dataset_root}')

IMAGE_SHAPE = (128, 128) # 자신의 데이터 셋에 맞추어서 조정!
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255,
                                                                  validation_split=0.2)
train_data = image_generator.flow_from_directory(dataset_root, target_size=IMAGE_SHAPE,
                                                 subset='training')
validation_data = image_generator.flow_from_directory(dataset_root, target_size=IMAGE_SHAPE,
                                                 subset='validation')

for image_batch, label_batch in validation_data:
    print(f'Image batch shape: {image_batch.shape}')
    print(f'Label batch shape: {label_batch.shape}')
    break

### 3. 학습 데이터 살펴보기

In [ ]:
classifier_url = 'https://tfhub.dev/google/imagenet/inception_v3/classification/4'

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,)) # Channel 3 RGB
])

labels_path = tf.keras.utils.get_file('ImageNetLabels.txt', 
                                      'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

In [ ]:
## using original ImangeNet classifier
result_batch = classifier.predict(image_batch)
print(f'Batch result shape: {result_batch.shape}')

predicted_class_names = imagenet_labels[np.argmax(result_batch, axis=-1)]
print(f'Batch predicted class names: {predicted_class_names}')

fig = plt.figure(figsize=(10, 10.5))
for n in range(30):
    ax = fig.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    ax.set_title(predicted_class_names[n])
    ax.axis('off')
_ = fig.suptitle('ImageNet predictions')

### 4. Convolution Neural Network

In [ ]:
## Log class
### https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_val_losses = []
        self.batch_acc = []
        self.batch_val_acc = []
    
    def on_epoch_end(self, epoch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['accuracy'])
        self.batch_val_losses.append(logs['val_loss'])
        self.batch_val_acc.append(logs['val_accuracy'])
        self.model.reset_metrics()

In [ ]:
cnn_model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=image_batch.shape[1:]),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(train_data.num_classes)])

cnn_model.summary()

### 5. Train model

In [ ]:
base_learning_rate = 0.001

cnn_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
                  loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                  metrics=['accuracy'])

In [ ]:
steps_per_epoch = np.ceil(train_data.samples/train_data.batch_size) # train all dataset per epoch
epochs = 25*2
cnn_callback = CollectBatchStats()

cnn_history = cnn_model.fit(train_data,
                            epochs=epochs,
                            steps_per_epoch=steps_per_epoch,
                            validation_data=validation_data,
                            callbacks=[cnn_callback])

### 6. Visulize results

In [ ]:
# Draw learning curves chart
acc = cnn_callback.batch_acc
val_acc = cnn_callback.batch_val_acc
loss = cnn_callback.batch_losses
val_loss = cnn_callback.batch_val_losses

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 1, 1)
ax.plot(acc, label='Training Accuracy')
ax.plot(val_acc, label='Validation Accuracy')
ax.legend(loc='lower right')
ax.set_ylabel('Accuracy')
ax.set_ylim([min(plt.ylim()),1])
ax.set_title('Training and Validation Accuracy')

ax = fig.add_subplot(2, 1, 2)
ax.plot(loss, label='Training Loss')
ax.plot(val_loss, label='Validation Loss')
ax.legend(loc='upper right')
ax.set_ylabel('Cross Entropy')
ax.set_ylim([0,1.0])
ax.set_title('Training and Validation Loss')
ax.set_xlabel('epoch')
_ = fig.suptitle('Our Convolution Neural Network')

In [ ]:
# Plot results
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(f'Classes: {class_names}')

## get result labels
predicted_batch = cnn_model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

## plot
fig = plt.figure(figsize=(10, 10.5))
for n in range(30):
    ax = fig.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    ax.set_title(predicted_label_batch[n].title(), color=color)
    ax.axis('off')
_ = fig.suptitle('Model predictions (green: correct, red: incorrect)')

### 7. Transfer Learning

In [ ]:
# Prepare transfer learning
feature_extractor_url = 'https://tfhub.dev/google/tf2-preview/inception_v3/feature_vector/4'
feature_extractor_layer = hub.KerasLayer(feature_extractor_url,
                                         input_shape=IMAGE_SHAPE+(3, ))
feature_batch = feature_extractor_layer(image_batch)
print(f'Feature vector shape: {feature_batch.shape}')

## Frozen feature extraction layer
feature_extractor_layer.trainable = False

In [ ]:
## Make a model for classification
model = tf.keras.Sequential([
    feature_extractor_layer,
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')
])

model.summary()

predictions= model(image_batch)
print(f'Prediction shape: {predictions.shape}')

### 8. Train transfered model

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate),
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [ ]:
steps_per_epoch = np.ceil(train_data.samples/train_data.batch_size) # train all dataset per epoch
initial_epoch = 25
batch_stats_callback = CollectBatchStats()

history = model.fit(train_data,
                    epochs=initial_epoch,
                    steps_per_epoch=steps_per_epoch,
                    validation_data=validation_data,
                    callbacks=[batch_stats_callback])

### 9. Visualize transfered model result

In [ ]:
# Draw learning curves chart
acc = batch_stats_callback.batch_acc
val_acc = batch_stats_callback.batch_val_acc
loss = batch_stats_callback.batch_losses
val_loss = batch_stats_callback.batch_val_losses

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 1, 1)
ax.plot(acc, label='Training Accuracy')
ax.plot(val_acc, label='Validation Accuracy')
ax.legend(loc='lower right')
ax.set_ylabel('Accuracy')
ax.set_ylim([min(plt.ylim()),1])
ax.set_title('Training and Validation Accuracy')

ax = fig.add_subplot(2, 1, 2)
ax.plot(loss, label='Training Loss')
ax.plot(val_loss, label='Validation Loss')
ax.legend(loc='upper right')
ax.set_ylabel('Cross Entropy')
ax.set_ylim([0,1.0])
ax.set_title('Training and Validation Loss')
ax.set_xlabel('epoch')
_ = fig.suptitle('Transfer Learning: Convolution Neural Network')

In [ ]:
# Plot results
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(f'Classes: {class_names}')

## get result labels
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

## plot
fig = plt.figure(figsize=(10, 10.5))
for n in range(30):
    ax = fig.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    ax.set_title(predicted_label_batch[n].title(), color=color)
    ax.axis('off')
_ = fig.suptitle('Model predictions (green: correct, red: incorrect)')

### 10. Transfer Learning Finetune

In [ ]:
## Unfrozen feature extraction layer
feature_extractor_layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=base_learning_rate/10),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy'])

model.summary()

In [ ]:
finetune_epoch = 25

history_fine = model.fit(train_data,
                        epochs=initial_epoch+finetune_epoch,
                        initial_epoch=initial_epoch,
                        steps_per_epoch=steps_per_epoch,
                        validation_data=validation_data,
                        callbacks = [batch_stats_callback])

### 11. Visualize finetune result

In [ ]:
# Draw learning curves chart
fine_acc = batch_stats_callback.batch_acc
fine_val_acc = batch_stats_callback.batch_val_acc
fine_loss = batch_stats_callback.batch_losses
fine_val_loss = batch_stats_callback.batch_val_losses

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(2, 1, 1)
ax.plot(acc, label='Training Accuracy')
ax.plot(val_acc, label='Validation Accuracy')
ax.set_ylabel('Accuracy')
ax.set_ylim([min(plt.ylim()),1])
ax.plot([initial_epoch,initial_epoch],
        ax.get_ylim(), label='Start Fine Tuning')
ax.legend(loc='lower right')
ax.set_title('Training and Validation Accuracy')

ax = fig.add_subplot(2, 1, 2)
ax.plot(loss, label='Training Loss')
ax.plot(val_loss, label='Validation Loss')
ax.set_ylabel('Cross Entropy')
ax.set_ylim([0,1.0])
ax.plot([initial_epoch,initial_epoch],
        ax.get_ylim(), label='Start Fine Tuning')
ax.legend(loc='upper right')
ax.set_title('Training and Validation Loss')
ax.set_xlabel('epoch')

In [ ]:
# Plot results
class_names = sorted(validation_data.class_indices.items(), key=lambda pair:pair[1])
class_names = np.array([key.title() for key, value in class_names])
print(f'Classes: {class_names}')

## get result labels
predicted_batch = model.predict(image_batch)
predicted_id = np.argmax(predicted_batch, axis=-1)
predicted_label_batch = class_names[predicted_id]

label_id = np.argmax(label_batch, axis=-1)

## plot
fig = plt.figure(figsize=(10, 10.5))
for n in range(30):
    ax = fig.add_subplot(6, 5, n+1)
    ax.imshow(image_batch[n])
    color = 'green' if predicted_id[n] == label_id[n] else 'red'
    ax.set_title(predicted_label_batch[n].title(), color=color)
    ax.axis('off')
_ = fig.suptitle('Model predictions (green: correct, red: incorrect)')

In [ ]:
idx = 0

test_data = validation_data[idx//32][0][idx%32]
actual_label = class_names[np.argmax(validation_data[idx//32][1][idx%32])]
predicted_label = class_names[np.argmax(model.predict(tf.expand_dims(test_data, 0)))]

fig = plt.figure(figsize=(6, 6))
fig.set_facecolor('white')
ax = fig.add_subplot()
axm = ax.imshow(test_data)
fig.suptitle(f'Test Image [{idx}]', fontsize=14)
ax.set_title(f'Label: {predicted_label} (Actual: {actual_label})', fontsize=12)
ax.grid(False)